## Training area for agents in Team Hunger Maze

In [6]:
# Imports
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import matplotlib.pyplot as plt
import maze_generator.maze_dataset as md
from Maze_env.reward_functions.maze_runner import MazeRunnerRewardsFun
import numpy as np
from DQN.training.basic import BaseTraining
from DQN.agents.hungermaze import HungerGamesAgent
from DQN.agents.basic import BaseAgent
from Maze_env.reward_functions.hunger_games import HungerGamesRewardsFun
from Maze_env.game_info.team_hunger_games_info import TeamHungerGames

import DQN.models.base as base
import os
print(os.getcwd())

/home/cody/Documents/DataSciBC/Generative_AI/maze_game/TrainingGround


### Define the Mazes

In [7]:
dataset = md.Maze_dataset(1,(3,3),maze_type = 'percolation')

In [8]:
# example of the maze
dataset.show_maze(0)

<IPython.core.display.Javascript object>

In [9]:
# parameters for the agents
n_agents = 1
vision = 3

In [10]:
%matplotlib notebook


# --- Agents for the Hunger Games --- #
rewards = rewards = HungerGamesRewardsFun(
                        GOAL = 100.0,
                         SEE_GOAL = 0.00,
                         DONT_SEE_GOAL = -0.00,
                         NEW_PLACE = 50.0,
                         OLD_PLACE = -0.75,
                         GET_CLOSER = 50.0, 
                         GET_CLOSER_CONSTANT = 50.0,
                         GET_FARTHER = -30.0,
                         GET_FARTHER_CONSTANT = -30.0,
                         DIST = 0.0,
                         HIT_OTHER = -100.0,
                         TOO_CLOSE = -1.00,
                         TOO_CLOSE_CONSTANT = -1.00,
                         NO_NEIGHBORS = 1.0
                )
game_info = TeamHungerGames(n_teams=2)
game_info.update_game_info(name='TeamTraining',
                           rewards_dist = {'team_0': rewards,
                                           'team_1': rewards},

)
maze_agent =BaseAgent({'team_0':base.CNN_version1,
                              'team_1': base.CNN_version1},
                               vision={'team_0':vision,
                                       'team_1': vision},
                               action_type='cardinal',
                               dist_paradigm='path',
                               game_info=game_info
                               )
maze_agent.load_object('team_1','TeamMazeRunner','team_0')

# --- trainer of the agent --- #
train = BaseTraining(name = 'OffHG_training',
                              maze_dataset = dataset,
                              maze_agent = maze_agent,
                              len_game=50,
                              n_objects={'team_0':n_agents,
                                         'team_1': n_agents},
                              final_epsilon = 0.05,
                              gamma = 0.99,
                              tau = 0.0001,
                               batch_size = 64,
                              n_frames = 500000,
                              lr = 0.0005,
                              lr_step_size=5000,
                              lr_gamma = 0.98,
                              lr_head_gamma = 0.98,
                              l2_regular=0.01,
                              replay_buffer_size=400000,
                              replay_buffer_min_perc=0.1,
                              target_update=5000,
                              policy_update=1,
                              lambda_entropy=0.1,
                              beta = 0.4,
                              alpha = 0.6,
                              decay_total = 400000,
                              per = True,
                              frame_mult=1.5,
                              )
train.training_toggle_off(['team_1'])
train.train(test_agent=False,peak=False)

/home/cody/Documents/DataSciBC/Generative_AI/maze_game/DQN/agents/basic.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  param_load = torch.load(os.path.join(object_fd,

Reward Distribution for object team_0
----------------Rewards Distribution --------------
HIT_OTHER: -100.0
TOO_CLOSE: -1.0
TOO_CLOSE_CONSTANT: -1.0
NO_NEIGHBORS: 1.0
SEE_GOAL: 0.0
DONT_SEE_GOAL: -0.0
NEW_PLACE: 50.0
OLD_PLACE: -0.75
GET_CLOSER: 50.0
GET_CLOSER_CONSTANT: 50.0
GET_FARTHER: -30.0
GET_FARTHER_CONSTANT: -30.0
DIST: 0.0
GOAL: 100.0
FAIL: -1.0
-----------------------------------------------------
Reward Distribution for object team_1
----------------Rewards Distribution --------------
HIT_OTHER: -100.0
TOO_CLOSE: -1.0
TOO_CLOSE_CONSTANT: -1.0
NO_NEIGHBORS: 1.0
SEE_GOAL: 0.0
DONT_SEE_GOAL: -0.0
NEW_PLACE: 50.0
OLD_PLACE: -0.75
GET_CLOSER: 50.0
GET_CLOSER_CONSTANT: 50.0
GET_FARTHER: -30.0
GET_FARTHER_CONSTANT: -30.0
DIST: 0.0
GOAL: 100.0
FAIL: -1.0
-----------------------------------------------------
----------------------------------
Basic epsilon decay scheduler:
Start epsilon: 1
End epsilon: 0.05
Decay total: 400000
Decay rate: 0.0007451821305330566

----------------------

KeyboardInterrupt: 

In [ ]:
train.save()
train.results()

/home/cody/Documents/DataSciBC/Generative_AI/maze_game/DQN/training/basic.py:651: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.lineplot(data = scores_df, x ='frame',y = 'score',ax = axe[1][1],palette='tab10')


<IPython.core.display.Javascript object>

In [ ]:
train.agents.test_agent(dataset,n_episodes=1000,len_game=15,num_objects={'team_0':n_agents,'team_1':n_agents})

/home/cody/miniconda3/envs/GameRFL/lib/python3.12/site-packages/gymnasium/utils/passive_env_checker.py:245: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'dict'>
  logger.warn(


{'team_0': 0.11500000000000009, 'team_1': 0.7880000000000006}

In [ ]:

train.agents.run_agent(dataset[0],num_objects={'team_0':n_agents, 'team_1':n_agents},n_episodes=20,len_game=15,epsilon=0,init_pos={})


/home/cody/miniconda3/envs/GameRFL/lib/python3.12/site-packages/gymnasium/utils/passive_env_checker.py:245: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'dict'>
  logger.warn(


cumulative reward: {'team_0': [-95.52283950617284], 'team_1': [-59.0216049382716]}
cumulative reward: {'team_0': [-84.72222222222221], 'team_1': [-59.0216049382716]}
cumulative reward: {'team_0': [-94.79979423868313], 'team_1': [-56.36625514403292]}
cumulative reward: {'team_0': [-94.79979423868313], 'team_1': [-57.03086419753086]}
cumulative reward: {'team_0': [-82.22037037037038], 'team_1': [-42.35493827160493]}
cumulative reward: {'team_0': [-92.49526748971194], 'team_1': [-58.23827160493826]}
cumulative reward: {'team_0': [-6.572222222222223], 'team_1': [-6.564197530864197]}
cumulative reward: {'team_0': [-90.27777777777777], 'team_1': [-56.45452674897119]}
cumulative reward: {'team_0': [-90.27777777777777], 'team_1': [-56.36625514403292]}
cumulative reward: {'team_0': [-89.91584362139916], 'team_1': [-58.141975308641975]}
cumulative reward: {'team_0': [-10.543827160493827], 'team_1': [-9.775308641975307]}
cumulative reward: {'team_0': [-95.83333333333334], 'team_1': [-56.454526748